In [ ]:
#default_exp data.datasets

# Datasets

> API details.

In [ ]:
#export
import logging
import requests
import subprocess
import zipfile
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, List, Optional, Tuple, Union

import numpy as np
import pandas as pd
from tqdm import tqdm

SOURCE_URL = 'https://robjhyndman.com/data/27-3-Athanasopoulos1.zip'
BASE_URL = 'https://github.com/FedericoGarza/meta-data/releases/download/vtourism/'

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Utils para descargar archivos

In [ ]:
#export
def download_file(directory: Union[str, Path], source_url: str, decompress: bool = False) -> None:
    """Download data from source_ulr inside directory.

    Parameters
    ----------
    directory: str, Path
        Custom directory where data will be downloaded.
    source_url: str
        URL where data is hosted.
    decompress: bool
        Wheter decompress downloaded file. Default False.
    """
    if isinstance(directory, str):
        directory = Path(directory)
    directory.mkdir(parents=True, exist_ok=True)

    filename = source_url.split('/')[-1]
    filepath = directory / filename

    # Streaming, so we can iterate over the response.
    r = requests.get(source_url, stream=True)
    # Total size in bytes.
    total_size = int(r.headers.get('content-length', 0))
    block_size = 1024 #1 Kibibyte

    t = tqdm(total=total_size, unit='iB', unit_scale=True)
    with open(filepath, 'wb') as f:
        for data in r.iter_content(block_size):
            t.update(len(data))
            f.write(data)
    t.close()

    if total_size != 0 and t.n != total_size:
        logger.error('ERROR, something went wrong downloading data')

    size = filepath.stat().st_size
    logger.info(f'Successfully downloaded {filename}, {size}, bytes.')

    if decompress:
        with zipfile.ZipFile(filepath, 'r') as zip_ref:
            zip_ref.extractall(directory)

        logger.info(f'Successfully decompressed {filepath}')

Todos los datasets se heredarían de esta dataclass que tiene los atributos que queremos

In [ ]:
#export
@dataclass
class TimeSeriesDataset:
    """
    Args:
        S (pd.DataFrame): Array of static features of shape
            (n_time_series, n_features).
        X (pd.DataFrame): Array of exogenous variables of shape
            (n_time_series, n_exogenous, n_periods).
        Y (pd.DataFrame): Array of target variable of shape
            (n_time_series, n_periods).
        idx_categorical_static (list, optional): List of categorical indexes
            of S.
        groups (dict, optional): Dictionary of groups. 
            Example: {'Yearly': ['Y1', 'Y2']}
    """
    S: np.ndarray
    X: np.ndarray
    Y: np.ndarray
    idx_categorical_static: Optional[List] = None
    groups: Optional[Dict[str, np.array]] = None

Información meta de Turismo

In [ ]:
#export
@dataclass
class Yearly:
    seasonality: int = 1
    horizon: int = 4
    freq: str = 'D'
    rows: int = 2
    name: str = 'Yearly'

@dataclass
class Quarterly:
    seasonality: int = 4
    horizon: int = 8
    freq: str = 'Q'
    rows: int = 3
    name: str = 'Quarterly'

@dataclass
class Monthly:
    seasonality: int = 12
    horizon: int = 24
    freq: str = 'M'
    rows: int = 3
    name: str = 'Monthly'

@dataclass
class TourismInfo:
    groups: Tuple = (Yearly, Quarterly, Monthly)

In [ ]:
#export
class Tourism(TimeSeriesDataset):
    
    @staticmethod
    def load(directory: str, training: bool = True, download: bool = True) -> 'Tourism':
        """
        Downloads and loads Tourism data.

        Parameters
        ----------
        directory: str
            Directory where data will be downloaded.
        training: bool
            Wheter return training or testing data. Default True.
        download: bool
            Wheter download dataset first.
        """
        path = Path(directory) / 'tourism' / 'datasets'
        
        if download:
            Tourism.download(directory)

        data = []
        groups = {}

        for group in TourismInfo.groups:
            if training:
                file = path / f'{group.name.lower()}_in.csv'
            else:
                file = path / f'{group.name.lower()}_oos.csv'

            df = pd.read_csv(file)
            groups[group.name] = df.columns.values

            dfs = []
            for col in df.columns:
                df_col = df[col]
                lenght = df_col[0].astype(int)
                skip_rows = group.rows

                df_col = df_col[skip_rows:lenght + skip_rows]
                df_col = df_col.rename('y').to_frame()
                df_col['unique_id'] = col

                dfs.append(df_col)

            df = pd.concat(dfs)
            df['ds'] = df.groupby('unique_id').cumcount() + 1

            data.append(df)

        data = pd.concat(data).reset_index(drop=True)[['unique_id', 'ds', 'y']]

        return Tourism(Y=data, S=None, X=None, groups=groups)

    @staticmethod
    def download(directory: str) -> None:
        """Download Tourism Dataset."""
        path = Path(directory) / 'tourism' / 'datasets'
        download_file(path, SOURCE_URL, decompress=True)
    
    def get_group(self, group: str) -> 'Tourism':
        """Filters group data.

        Parameters
        ----------
        group: str
            Group name.
        """
        assert group in self.groups, \
            f'Please provide a valid group: {", ".join(self.groups.keys())}'

        ids = self.groups[group]

        y = self.Y.query('unique_id in @ids')

        return Tourism(Y=y, S=None, X=None, groups={group: ids})

In [ ]:
tourism_dataset = Tourism.load('data')

100%|██████████| 356k/356k [00:00<00:00, 1.46MiB/s]
INFO:__main__:Successfully downloaded 27-3-Athanasopoulos1.zip, 356334, bytes.
INFO:__main__:Successfully decompressed data/tourism/datasets/27-3-Athanasopoulos1.zip


Si se necesita un grupo en particular se usa el método `get_group`.

In [ ]:
tourism_dataset_yearly = tourism_dataset.get_group('Yearly')

In [ ]:
tourism_dataset_yearly.Y.head()

,unique_id,ds,y
0,Y1,1,25092.2284
1,Y1,2,24271.5134
2,Y1,3,25828.9883
3,Y1,4,27697.5047
4,Y1,5,27956.2276
